In [45]:
import asyncio
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

True

<h3>GraphQL Query Creator</h3>

In [135]:
import re

def generate_yelp_graphql_queries(search_locations, max_locations_per_query=10):
    """
    Generates a list of GraphQL queries for the Yelp API based on a list of search locations,
    with a maximum of `max_locations_per_query` locations per query.

    Parameters:
    - search_locations (list): A list of dictionaries, each containing 'name', 'address', 'city', 'state', and 'country' keys.
    - max_locations_per_query (int): The maximum number of locations per query.

    Returns:
    - list: A list of GraphQL query strings for the provided search locations.
    """
    # Split search_locations into chunks of max_locations_per_query
    chunks = [search_locations[i:i + max_locations_per_query] for i in range(0, len(search_locations), max_locations_per_query)]
    
    queries = []
    for chunk_index, chunk in enumerate(chunks):
        query_parts = [f"query MyQuery{chunk_index}{{"]
        for index, location in enumerate(chunk):
            # Sanitize the name by removing spaces, special characters, and non-ASCII characters
            sanitized_name = re.sub(r'[^\x20-\x7E\s]', '', location['name'])
            print(sanitized_name)
            sanitized_sanitized_name = re.sub(r'[^a-zA-Z]', '', location['name'])
            # Ensure the address and other strings are properly escaped
            # Construct the business_match query part
            query_parts.append(
                f"{sanitized_sanitized_name}: business_match("
                f"address1: \"{location['address']}\", "
                f"city: \"{location['city']}\", "
                f"country: \"{location['country']}\", "
                f"name: \"{sanitized_name}\", "
                f"state: \"{location['state']}\""
                ") {"
                "  business {"
                "    name"
                "    photos"
                "    rating"
                "    coordinates {"
                "      latitude"
                "      longitude"
                "    }"
                "  }"
                "}"
            )
        query_parts.append("}")
        query = " ".join(query_parts)
        queries.append(query)
    return queries

<h3>Yelp API Output Parser</h3>

In [102]:
def parse_yelp_response(response_text):
    """
    Parses the Yelp GraphQL API response and formats it into a list of dictionaries.

    Parameters:
    - response_text (str): The JSON response text from the Yelp API.

    Returns:
    - list: A list of dictionaries, each containing 'name', 'address', 'photo', 'rating', 'lat', and 'lon'.
    """

    formatted_results = []
    print(response_text)

    for search_key, search_result in response_text.items():
        for business in search_result.get('business', []):
            formatted_results.append({
                'name': business.get('name'),
                'photo': business['photos'][0] if business['photos'] else None,
                'rating': business.get('rating'),
                'lat': business['coordinates'].get('latitude'),
                'lon': business['coordinates'].get('longitude')
            })

    return formatted_results

<h3>Yelp API BATCH Wrapper</h3>

<h3>Yelp API Location Wrapper</h3>

In [49]:
async def yelp_wrapper(location, term="attractions", radius=20000, sort_by="best_match", limit=20):
    # Get a list of locations from Yelp API
    params = {
        "location": location,
        "term": term,
        "radius": radius,
        "sort_by": sort_by,
        "limit": limit
    }
    yelp_url = f"https://api.yelp.com/v3/businesses/search?{urllib.parse.urlencode(params)}"
    yelp_header = {
        "accept": "application/json",
        "Authorization": f"Bearer {os.getenv('YELP_API_KEY')}"
    }
    response = await asyncio.to_thread(requests.get, yelp_url, headers=yelp_header)
    if response.status_code == 200:
        yelp_data = response.json()
        # Parse and format the response
        formatted_locations = [
            {
                "name": business.get("name"),
                "address": " ".join(business["location"].get("display_address", [])),
                "photo": business.get("image_url"),
                "rating": business.get("rating"),
                "description": ", ".join(category["title"] for category in business.get("categories", [])),
                "lat": business["coordinates"].get("latitude"),
                "lon": business["coordinates"].get("longitude")
            }
            for business in yelp_data.get("businesses", [])
        ]
        return json.dumps(formatted_locations)
    
    else:
        response.raise_for_status()

<h3>OpenAI Wrapper</h3>

In [50]:
from openai import AsyncOpenAI

async def openai_wrapper(location, preferences = ""):
    # Get a list of 10 locations form OpenAI api
    openai_client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    print(location, preferences)
    client_content = f"""
I am traveling to {location}, and I want to visit attractions around this area that fit these criteria: {preferences}.
Generate a list of 10 attraction objects and return in this format:
{{
    "attractions": [
        {{
            "name": "<name>",
            "address": "<street address>",
            "city": "<city>",
            "state": "<state>",
            "country": "<country>",
        }}
    ]
}}
"""
    chat_completion = await openai_client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "You are a helpful travel assistant designed to output JSON containing a list of objects that have atrribute name and address."},
            {"role": "user", "content": client_content}
        ],
    )
    return json.loads(chat_completion.choices[0].message.content)["attractions"]

In [81]:
locations = [
    "Fremont, CA",
    "Berkeley, CA",
    "Irvine, CA",
    "New York, NY",
]

preferences = "tech"

openai_tasks = [openai_wrapper(location, preferences) for location in locations]
openai_results = await asyncio.gather(*openai_tasks)
    
print("Openai Results:")
print(openai_results)
    
# Combine the results from all locations
combined_openai_results = [attraction for result in openai_results for attraction in result]
print("Combined OpenAI Results:")
print(len(combined_openai_results))
print(combined_openai_results)



Fremont, CA tech
Berkeley, CA tech
Irvine, CA tech
New York, NY tech
Openai Results:
[[{'name': 'Tesla Factory Tour', 'address': '45500 Fremont Blvd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Computer History Museum', 'address': '1401 N Shoreline Blvd', 'city': 'Mountain View', 'state': 'CA', 'country': 'USA'}, {'name': 'Intel Museum', 'address': '2200 Mission College Blvd', 'city': 'Santa Clara', 'state': 'CA', 'country': 'USA'}, {'name': 'NASA Ames Research Center', 'address': 'Moffett Field', 'city': 'Mountain View', 'state': 'CA', 'country': 'USA'}, {'name': 'Googleplex', 'address': '1600 Amphitheatre Pkwy', 'city': 'Mountain View', 'state': 'CA', 'country': 'USA'}, {'name': 'Apple Park Visitor Center', 'address': '10600 N Tantau Ave', 'city': 'Cupertino', 'state': 'CA', 'country': 'USA'}, {'name': 'Facebook HQ', 'address': '1 Hacker Way', 'city': 'Menlo Park', 'state': 'CA', 'country': 'USA'}, {'name': 'The Tech Interactive', 'address': '201 S Market St', 'ci

In [125]:
import asyncio
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
import os

# Ensure you have the generate_yelp_graphql_queries and parse_yelp_response functions defined as before

async def execute_graphql_query(query_string, headers):
    transport = AIOHTTPTransport(
        url="https://api.yelp.com/v3/graphql",
        headers=headers,
    )
    # Create a GraphQL client using the defined transport
    client = Client(transport=transport, fetch_schema_from_transport=True)
    # Execute the query asynchronously
    query = gql(query_string)
    result = await client.execute_async(query)
    # Close the client after the query is executed
    await client.close_async()
    return result

async def yelp_batch_wrapper(search_locations):
    """
    Fetches data from Yelp API asynchronously using GraphQL.

    Parameters:
    - search_locations (list): A list of dictionaries with 'name' and 'address' for search locations.

    Returns:
    - list: A list of parsed JSON responses from the Yelp API.
    """
    # Generate all GraphQL queries
    graphql_queries = generate_yelp_graphql_queries(search_locations)
    print(graphql_queries)
    
    headers = {
        "Authorization": f"Bearer {os.getenv('YELP_API_KEY')}",
        "Content-Type": "application/json",
    }

    # List to store all the parsed results
    all_parsed_results = []

     # Execute each query sequentially with a one-second delay between them
    for query in graphql_queries:
        result = await execute_graphql_query(query, headers)
        parsed_results = parse_yelp_response(result)
        all_parsed_results.extend(parsed_results)
    
    return all_parsed_results

# Example usage:
# async def main():
#     search_locations = [
#         {"name": "SanJose", "address": "San Jose", "city": "San Jose", "state": "CA", "country": "USA"},
#         {"name": "SanFran", "address": "San Francisco", "city": "San Francisco", "state": "CA", "country": "USA"},
#         # ... more locations ...
#     ]
#     yelp_data = await yelp_batch_wrapper(search_locations)
#     print(yelp_data)

# asyncio.run(main())

In [133]:
# Feed the combined results into yelp_batch_wrapper
yelp_results = await yelp_batch_wrapper(combined_openai_results)
print("Yelp Results:")
print(yelp_results)

Tesla Factory Tour
Computer History Museum
Intel Museum
NASA Ames Research Center
Googleplex
Apple Park Visitor Center
Facebook HQ
The Tech Interactive
Silicon Valley Innovation Center
Autodesk Gallery
Lawrence Hall of Science
Berkeley Marina
Computer History Museum
Exploratorium
Intel Museum
Silicon Valley Innovation Center
Tech Interactive
UC Berkeley College of Engineering
Autodesk Gallery
Googleplex
Irvine Spectrum Center
Marconi Automotive Museum
Orange County Great Park
The Irvine Museum
Google Irvine
Blizzard Entertainment
UCI Beall Applied Innovation
Twitch Irvine
Anduril Industries
Amazon Irvine
The National 911 Memorial  Museum
The Museum of Modern Art MoMA
American Museum of Natural History
One World Observatory
Empire State Building
The High Line
Statue of Liberty National Monument
Brooklyn Bridge
Central Park
Times Square
['query MyQuery0{ TeslaFactoryTour: business_match(address1: "", city: "Fremont", country: "USA", name: "Tesla Factory Tour", state: "CA") {  business { 

<h3>Attractions Wrapper</h3>

In [93]:
async def attractions_wrapper(location, preferences=""):
    # Get results from OpenAI
    openai_result = await openai_wrapper(location, preferences)
        
    # Feed addresses from openai_result back into yelp_wrapper
    yelp_results = await yelp_batch_wrapper(openai_result)
    print(yelp_results)

In [94]:
# async def attractions_wrapper(locations, preferences=""):
#     # Get results from OpenAI
#     for location in locations:
#         openai_result = await openai_wrapper(location, preferences)
        
#     # Feed addresses from openai_result back into yelp_wrapper
#     yelp_results = await yelp_batch_wrapper(openai_result)
    
#     print(yelp_results)

async def attractions_wrapper(locations, preferences=""):
    # Run openai_wrapper concurrently for all locations
    openai_tasks = [openai_wrapper(location, preferences) for location in locations]
    openai_results = await asyncio.gather(*openai_tasks)
    
    print("Openai Results:")
    print(openai_results)
    
    # Combine the results from all locations
    combined_openai_results = [attraction for result in openai_results for attraction in result]
    print("Combined OpenAI Results:")
    print(len(combined_openai_results))
    print(combined_openai_results)
    
    # Feed the combined results into yelp_batch_wrapper
    yelp_results = await yelp_batch_wrapper(combined_openai_results)
    print("Yelp Results:")
    print(yelp_results)

In [95]:
import nest_asyncio
nest_asyncio.apply()

In [136]:
# await attractions_wrapper(["San Francisco, CA"], "museums")
cities = [
    "Fremont, CA",
    "Berkeley, CA",
    "Irvine, CA",
    "New York, NY",
    "Washington, DC",
    "Chicago, IL",
]

await attractions_wrapper(cities, "boba tea")

Fremont, CA boba tea
Berkeley, CA boba tea
Irvine, CA boba tea
New York, NY boba tea
Washington, DC boba tea
Chicago, IL boba tea
Openai Results:
[[{'name': 'T4', 'address': '43421 Christy St', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Happy Lemon', 'address': '46827 Warm Springs Blvd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Gong Cha', 'address': '39230 Argonaut Way', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Tea Island', 'address': '46132 Warm Springs Blvd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Sharetea', 'address': '3944 Washington Blvd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'One Tea', 'address': '46873 Warm Springs Blvd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Super Cue Cafe', 'address': '43773 Boscell Rd', 'city': 'Fremont', 'state': 'CA', 'country': 'USA'}, {'name': 'Teaspoon', 'address': '41093 Fremont Blvd', 'city': 'Fremont', 'state': 'CA'